# MNIST LSTM

In [33]:
'''
Loading necessary libraries
'''
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [34]:
'''
Setup parameters
'''
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

n_epochs = 10
n_classes = 10
batch_size = 100
lr = 1e-3

squence_length = 28
input_size = 28
hidden_size = 128
n_layers = 2

In [35]:
'''
Loading MNIST dataset
'''
# Datasets
train_dataset = torchvision.datasets.MNIST(root='./data/mnist', 
                                           train=True,
                                           download=True, 
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='./data/mnist', 
                                          train=False,
                                          download=True, 
                                          transform=transforms.ToTensor())

# Loaders
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=batch_size,
                                           shuffle=True, 
                                           num_workers=12)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False, 
                                          num_workers=12)

In [36]:
'''
Define model class
'''
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, n_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_size, hidden_size, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_classes)
        
    def forward(self, x): # (batch_size,1,28,28)
        x = x.squeeze() # (batch_size,28,28)
        
        # set initial hidden and cell states
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_size).to(device)
        
        # forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0)) # out: (batch_size, seq_len, hidden_size)
        
        # decode the hidden state of the time step
        out = self.fc(out[:, -1, :])
        return out

model = RNN(input_size, hidden_size, n_layers, n_classes).to(device)

In [37]:
'''
Optimizer and Loss function
'''
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [38]:
'''
Train the model
'''
total_step = len(train_loader)
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device) 
        labels = labels.to(device)
        
        # forward
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%500==0:
            print('Epoch [{}/{}]-[{}/{}] Loss: {}'
                 .format(epoch+1, n_epochs, i+1, total_step, loss.item()))
        

Epoch [1/10]-[500/600] Loss: 0.1681322455406189
Epoch [2/10]-[500/600] Loss: 0.10740723460912704
Epoch [3/10]-[500/600] Loss: 0.020998071879148483
Epoch [4/10]-[500/600] Loss: 0.06228170916438103
Epoch [5/10]-[500/600] Loss: 0.10480879992246628
Epoch [6/10]-[500/600] Loss: 0.015044903382658958
Epoch [7/10]-[500/600] Loss: 0.004647936671972275
Epoch [8/10]-[500/600] Loss: 0.04443446174263954
Epoch [9/10]-[500/600] Loss: 0.02333558164536953
Epoch [10/10]-[500/600] Loss: 0.008905739523470402


In [39]:
'''
Test the model
'''
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the model on test data: {}%'.format(100*correct/total))


Accuracy of the model on test data: 10.2%
